In [9]:
"""ESTABLISH A CONNECTION WITH THE DEVICES"""
import vna
IP = "192.168.3.51"
timeout = 25e3
max_npoints = 1e3
instr_vna = vna.ManageVNA(IP,timeout,max_npoints)
import sma
resource = 'TCPIP0::192.168.3.2::INSTR'
instr_sma = sma.ManageSMA(resource)
import sim
resource = 'ASRL34::INSTR'
instr_sim = sim.ManageSIM(resource)

In [2]:
import time
import numpy as np
from many_instruments import *
import h5py

In [3]:
"""Prepare VNA"""
port = "S21"
IFBW = 1e2 #[Hz]
powerdBm = -15 #[dBm]

instr_vna.reset()
instr_vna.set_mode("NA")
time.sleep(1)
instr_vna.set_port(port)
time.sleep(1)
instr_vna.set_IFBW(IFBW)

In [4]:
"""Prepare SMA (Pump)"""
instr_sma.set_output(1) # Turn on Pump
"""Prepare SIM (Bias)"""
#R = 289.5   #[Ohm]   #Resistance to calculate DC current from the applied voltage
R = 433.9 #[ +-1 Ohm ] #Con multimetro e verifica con V/I
instr_sim.set_voltage(round(R*0.76e-3,3))    # Set Bias to 1.5mA
instr_sim.set_output(1) # Turn on Bias

In [10]:
"""SCAN (VNA) parameters """
# The scan params are specified in each dataset name

fmin = 4e9
fmax = 8e9
npoints = 401
navgs = 1

#acquire_S21(instr_vna,fmin,fmax,powerdBm,npoints,navgs,"Test3.h5","/1a")
#freq, I, Q = instr_vna.single_scan(fmin,fmax,powerdBm,npoints,navgs)

In [11]:
"""Choose where SAVE results"""
filename = "03_03_4-8Ghz_15pts.h5"

In [12]:
"""Simulation parameters"""
# Pump
freq_start = 15e9
freq_stop = 15.75e9
ampl_min = -20
ampl_max = -20
# Bias
i_min = 0.76e-3
i_max = 0.76e-3
volt_min = R*i_min
volt_max = R*i_max

# Points densities
n_freq = 15
n_ampl = 1
n_volt = 1

freqs = np.linspace(freq_start, freq_stop, n_freq) 
ampls = np.linspace(ampl_min, ampl_max, n_ampl)
volts = np.linspace(volt_min, volt_max, n_volt)

In [13]:
# Save simulation parameters!
name_array = ["powerdBm","freq_start","freq_stop","ampl_min","ampl_max","volt_min","volt_max","n_freq","n_ampl","n_volt","navgs"]
value_array = [str(powerdBm),str(freq_start),str(freq_stop),str(ampl_min),str(ampl_max),str(volt_min),str(volt_max),str(n_freq),str(n_ampl),str(n_volt),str(navgs)]
hf = h5py.File(filename, 'a')
mat = [name_array, value_array]
hf.create_dataset("parameters", data = mat)
hf.close()


"""SIMULATION"""
wait_meter = 0.1
for k in np.arange(n_volt) :
    time.sleep(wait_meter)
    instr_sim.set_voltage(round(volts[k],3))
    for j in np.arange(n_ampl) :
        time.sleep(wait_meter)
        instr_sma.set_amplitude(ampls[j])
        for i in np.arange(n_freq) :
            time.sleep(wait_meter)
            instr_sma.set_freq(freqs[i])
            group = "dc_" + str(k) + "/pumpfreq_" + str(i) + "/pumppow_" + str(j)
            #freq, I, Q = instr_vna.single_scan(fmin,fmax,powerdBm,npoints,navgs)
            acquire_S21(instr_vna,fmin,fmax,powerdBm,npoints,navgs,filename,group)
            

            # #Write
            # Mat = [freq,I,Q]
            # dataset_name = "dataset_" + str(index)
            # hf.create_dataset(dataset_name, data=Mat)
            # index = index + 1

"""Turn OFF Pump and Bias"""
instr_sma.set_output(0) # Pump Off
instr_sim.set_output(0) # Bias Off